# AdWiseAI – Data Cleaning & Feature Engineering (Sprint 2)

This notebook performs data cleaning and feature engineering on the raw ad campaign dataset (`adwiseai.ads_dataset_raw`), and saves a cleaned, enriched version as `adwiseai.ads_dataset_cleaned`.


In [0]:
from pyspark.sql import functions as F

# Set up schema
spark.sql("CREATE DATABASE IF NOT EXISTS adwiseai")
spark.catalog.setCurrentDatabase("adwiseai")

#load data
df = spark.table("adwiseai.ads_dataset_raw")

#veiw data
display(df)
df.printSchema()

In [0]:
# Cleanign string to numeric fields
df_clean = df.withColumn(
    "Acquisition_Cost_Clean", F.regexp_replace("Acquisition_Cost", "[$,]", "").cast("double")
).withColumn(
    "Duration_Days", F.regexp_extract("Duration", r"(\d+)", 1).cast("int")
)

# Check schema
df_clean.printSchema()    

In [0]:
# Feature Creation

# click through rate i.e % of people who clicked after seeing the ad
df_features = df_clean.withColumn("CTR", F.col("Clicks") / F.col("Impressions"))

# CPC (Cost Per Click) i.e How much did each click cost
df_features = df_features.withColumn("CPC", F.col("Acquisition_Cost_Clean") / F.col("Clicks"))

# CPM (Cost Per Mille) i.e cost per 1000 impressions
df_features = df_features.withColumn("CPM", (F.col("Acquisition_Cost_Clean") / F.col("Impressions")) * 1000)

# Campaign Week 
df_features = df_features.withColumn("Campaign_Week", F.weekofyear("Date"))

# Campaign Month
df_features = df_features.withColumn("Campaign_Month", F.month("Date"))

# Standardize the ads channel (INstAgram → instagram)
df_features = df_features.withColumn("Channel_Lower", F.lower(F.col("Channel_Used")))


Incomplete or zero-value records can skew your averages, break calculations, and confuse models. Clean data = reliable metrics.

In [0]:
#  Missing Values
df_filtered = df_features.dropna(subset=["Clicks", "Impressions", "Acquisition_Cost_Clean", "ROI", "Location", "Language", "Customer_Segment", "Campaign_Goal", "Channel_Lower", "Duration_Days", "Campaign_Week", "Campaign_Month"])

# Outtlier removal
df_filtered = df_filtered.filter((F.col("Impressions") > 0) & (F.col("Clicks") > 0))



In [0]:
# write to delta table
df_filtered.write.format("delta").mode("overwrite").saveAsTable("adwiseai.ads_dataset_cleaned")


### EDA: Basic Distribution Checks + Outlier Detection

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert filtered Spark DataFrame to Pandas for plotting
df_pd = df_filtered.select(
    "CTR", "CPC", "CPM", "ROI", "Conversion_Rate", "Engagement_Score"
).toPandas()



In [0]:
# Set consistent style
sns.set(style="whitegrid")

# Plot multiple features
features = ["CTR", "CPC", "CPM", "ROI", "Conversion_Rate", "Engagement_Score"]
for feature in features:
    plt.figure(figsize=(6, 4))
    sns.histplot(df_pd[feature], bins=40, kde=True)
    plt.title(f"{feature} Distribution")
    plt.xlabel(feature)
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()


#### Findings

##### CTR
Observation: The distribution is right-skewed with a strong peak around 0.325 and a smaller peak near 0.30.

Insight: Most ads have a high CTR, which could indicate effective ad targeting or optimization. The second peak could represent a different segment or campaign strategy.


##### CPC
Observation: Bimodal distribution with prominent peaks at ~0.4 and ~0.65.

Insight: There may be two distinct bidding strategies or campaign types — one more cost-efficient and another more aggressive or competitive

#### CPM
Observation: Also bimodal, with sharp peaks at ~130 and ~200.

Insight: This could indicate different ad placements or audience segments — one cheaper and another more premium.

#### ROI
Observation: Highly right-skewed, with most values between 0–2, then a flat tail reaching up to 8.

Insight: Most campaigns yield modest returns, but some perform significantly better. There may be a few "super-performers" driving this long tail.


#### Conversion Rate Distribution
Observation: Unusual repetitive peak pattern (looks almost periodic).

Insight: This likely indicates discrete values or rounding, maybe due to how conversion rates are recorded (e.g., calculated as fixed % steps). It may also suggest an issue in how the data is binned or generated.


#### Engagement Score Distribution
Observation: Heavily skewed with a massive spike at 1, then evenly spaced smaller peaks from 2 to 10.

Insight: A large proportion of users had minimal engagement, but a decent number scored evenly across the rest. Suggests many users drop off quickly while others engage more steadily.

In [0]:
from scipy.stats import zscore

# Add Z-scores to help identify extreme outliers
z_scores = df_pd[["CTR", "CPC", "CPM", "ROI"]].apply(zscore)
outlier_mask = (z_scores.abs() > 4)  # typical threshold
outliers = df_pd[outlier_mask.any(axis=1)]

print(f"Outlier rows detected: {len(outliers)}")
display(outliers)


In [0]:
# Group by Company and Channel to count number of campaigns
campaign_counts = df_filtered.groupBy("Company", "Channel_Used") \
    .count() \
    .orderBy("Company", "count", ascending=False)

display(campaign_counts)

In [0]:
pivot_view = df_filtered.groupBy("Company").pivot("Channel_Used").count().fillna(0)
display(pivot_view)
